We will create a math model of a JSP and solve it using a state of the art math. sovler
The most likely one will be PULP? \

https://github.com/KevinLu43/JSP-by-using-Mathematical-Programming-in-Python/blob/master/Python%2BPulp%E5%9F%BA%E6%9C%AC%E6%9E%B6%E6%A7%8B.md \ 

https://github.com/Eason0227/Job-shop-scheduling-problem/blob/main/PULP%20for%20JSSP.ipynb
https://www.supplychaindataanalytics.com/flow-shop-scheduling-with-pulp-in-python/

Other Options would be Python MIP with https://python-mip.readthedocs.io/en/latest/examples.html#:~:text=The%20Job%20Shop%20Scheduling%20Problem,use%20each%20machine%20only%20once. 
or pyomo - https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/04.03-Job-Shop-Scheduling.html

https://github.com/bruscalia/jobshop/blob/master/jobshop/mip/disjunctive.py

In [ ]:
!pip install pulp

$$
\begin{align}
\text{start}_{j,m} & \geq 0\\
\text{start}_{j,m}+\text{dur}_{j,m} & \leq \text{makespan}
\end{align}
$$

In [ ]:
%%time
import pandas as pd
import numpy as np
import time
import pulp 

job_num = 10
processing_df= pd.read_csv('processing_times.csv')
sequence_df = pd.read_csv('machine_sequence.csv')

processing_df = processing_df.drop(['Job'],axis=1)
sequence_df = sequence_df.drop(['Job'],axis=1)


pt = processing_df.values
mo_0 = sequence_df.values
mo = mo_0-1

dv = pulp.LpVariable.dicts("start_time",
                                     ((i, j) for i in range(job_num) for j in range(job_num)),
                                     lowBound=0,
                                     cat='Continuous')

b =  pulp.LpVariable.dicts("binary_var",
                                     ((i, j,o) for i in range(job_num) for j in range(job_num) for o in range(job_num) if j<o),
                                     lowBound=0,
                                     cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)
model += Cmax

for j in range(job_num):
    for i in range(job_num-1):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[J][list(mo[J]).index(I)]

for j in range(job_num):
    for i in range(job_num):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[J][list(mo[J]).index(I)]

for i in range(job_num):
    for j in range(job_num):
        for l in range(job_num):
            if j<l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[L][list(mo[L]).index(I)] - 1000*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[J][list(mo[J]).index(I)] - 1000*(b[i,J,L]))
       
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print( pulp.LpStatus[model.status] )

In [ ]:
# maybe construct the Varaibles with itertools? 
import itertools as iter

x    = {(i,j): pulp.LpVariable(f"x{i}{j}", 0,1, pulp.LpBinary) for i,j in iter.product(I, J)}
c    = {(j,k): pulp.LpVariable(f"c{j}{k}", 0, None, pulp.LpContinuous) for j,k in iter.product(J, K)}
maxi = {(j,k): pulp.LpVariable(f"maxi{j}{k}", 0, None, pulp.LpContinuous) for j,k in iter.product(J, K)}

# just make sure that I and J get new values